<img src="../../img/python-logo-no-text.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;">
  <b>Clean Code: Functions</b>
</div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>
<br/>
<div style="text-align:center;">module_230_clean_code/topic_140_a3_functions</div>


# Clean Code: Functions

Package meaningful operations as carefully named functions

- More readable
- Easier to test
- More likely to be reused
- Less likely to contain errors


## The first rule about functions

- Functions should be small
- Even smaller than that!
- No more than 4 lines!


## More relaxed rules

(From the C++ Core Guidelines)

- Functions should fit on a screen
- Break large functions up into smaller cohesive and named functions
- One-to-five-lines functions should be considered normal


## Do one thing

- Functions should do one thing
- They should do it well
- They should do it only


## Abstraction levels

Everything the function does in its body should be one (and only one) level of
abstraction below the function itself.


## "To" paragraphs: Checking levels of abstraction

To render_page_with_setups_and_teardowns

We check to see whether the page is a test page and

if so, we include the setups and teardowns.

In either case we render the page in HTML


## The Step-Down Rule

- We want code to read like a top-down narrative
- Every function should be followed by those one level of abstraction below it


## Mini workshop: Do one thing

The function `handle_money_stuff()` does more than one thing.

Split it into several functions so that each does one thing only. Ensure that
- each function does its job well and is at a single level of abstraction,
- all names are appropriate, and
- the code is easy to understand.

*Hint:* Start by renaming the variables according to the comments to simplify
the rest of the work.


The function `handle_money_stuff()` has the following parameters:

- the day of the week (`i_dow`),
- the salary per day (`f_spd`),
- the name of the employee (`str_n`) and
- a list of the salaries paid so far (`lst_slrs`).

The new salary is appended to the list `lst_slrs`.

The function returns the tax to be paid.

In [ ]:
def handle_money_stuff(i_dow: int, f_spd: float, str_n: str, lst_slrs: list):
    # Get the day of week from the list of days.
    # We count Sunday as 1, Monday as 2, etc. but the work week starts on Monday.
    str_dow = lst_dns[i_dow - 1]
    # Compute the salary so far based on the day
    f_ssf = (i_dow - 1) * f_spd
    # The tax
    f_t = 0.0
    if f_ssf > 500.0 and f_ssf <= 1000.0:
        f_t = f_ssf * 0.05
    elif f_ssf > 1000.0 and f_ssf <= 2000.0:
        f_t = f_ssf * 0.1
    else:
        f_t = f_ssf * 0.15
    # Update salary based on the tax to pay
    f_ssf = f_ssf - f_t
    # Add the salary to the list of all salaries paid
    lst_slrs.append(f_ssf)
    print(f"{str_n} worked till {str_dow} and earned ${f_ssf} this week.")
    print(f"Their taxes were ${f_t}.")
    return f_t

In [ ]:
_salaries = [2345, 1234]
_result = handle_money_stuff(4, 200, "Joe", _salaries)
print(_result)

In [ ]:
assert _salaries == [2345, 1234, 570]
assert _result == 30

In [ ]:
_salaries = [2345, 1234]
_result = compute_salary_and_taxes_v1(4, 200, "Joe", _salaries)
print(_result)

In [ ]:
assert _salaries == [2345, 1234, 570]
assert _result == 30

In [ ]:
_salaries = [2345, 1234]
_result = compute_salary_and_taxes_v2(4, 200, "Joe", _salaries)
print(_result)

In [ ]:
assert _salaries == [2345, 1234, 570]
assert _result == 30


## Switches and abstraction

Switch statements often perform operations on the same level of abstraction
(for “subtypes” instead of the original type).

“Subtypes” are often distinguished by type tags

In [ ]:
from enum import IntEnum  # noqa: E402
from dataclasses import dataclass  # noqa: E402

In [ ]:
class EmployeeType(IntEnum):
    COMMISSIONED = 0
    HOURLY = 1
    SALARIED = 2

In [ ]:
@dataclass
class Money:
    amount_in_euros: float

    def __add__(self, rhs):
        return Money(self.amount_in_euros + rhs.amount_in_euros)

In [ ]:
print(Money(100.0) + Money(50.0))

In [ ]:
@dataclass
class EmployeeV1:
    type: EmployeeType

In [ ]:
def calculate_commissioned_pay(e: EmployeeV1):
    return Money(100.0)

In [ ]:
def calculate_hourly_pay(e: EmployeeV1):
    return Money(120.0)

In [ ]:
def calculate_salaried_pay(e: EmployeeV1):
    return Money(80.0)

In [ ]:
def calculate_pay(e: EmployeeV1):
    if e.type == EmployeeType.COMMISSIONED:
        return calculate_commissioned_pay(e)
    elif e.type == EmployeeType.HOURLY:
        return calculate_hourly_pay(e)
    elif e.type == EmployeeType.SALARIED:
        return calculate_salaried_pay(e)
    else:
        raise ValueError("No valid employee type.")

In [ ]:
e1 = EmployeeV1(type=EmployeeType.HOURLY)
e2 = EmployeeV1(EmployeeType.SALARIED)

In [ ]:
print(calculate_pay(e1))
print(calculate_pay(e2))


## Replace switch with polymorphism

It is often better to replace switches with “real” subtyping and polymorphism:

In [ ]:
from abc import ABC, abstractmethod  # noqa: E402

In [ ]:
class Employee(ABC):
    @abstractmethod
    def calculate_pay(self):
        ...

In [ ]:
@dataclass
class CommissionedEmployee(Employee):
    def calculate_pay(self):
        return Money(100.0)

In [ ]:
@dataclass
class HourlyEmployee(Employee):
    def calculate_pay(self):
        return Money(120.0)

In [ ]:
@dataclass
class SalariedEmployee(Employee):
    def calculate_pay(self):
        return Money(80.0)

In [ ]:
def create_employee(employee_type: EmployeeType):
    if employee_type == EmployeeType.COMMISSIONED:
        return CommissionedEmployee()
    elif employee_type == EmployeeType.HOURLY:
        return HourlyEmployee()
    elif employee_type == EmployeeType.SALARIED:
        return SalariedEmployee()
    else:
        raise ValueError("Not a valid employee type.")

In [ ]:
e1 = create_employee(EmployeeType.HOURLY)
e2 = create_employee(EmployeeType.SALARIED)

In [ ]:
print(e1.calculate_pay())
print(e2.calculate_pay())

In [ ]:
class EmployeeFactory:
    def create_employee(self, employee_type: EmployeeType):
        if employee_type == EmployeeType.COMMISSIONED:
            return CommissionedEmployee()
        elif employee_type == EmployeeType.HOURLY:
            return HourlyEmployee()
        elif employee_type == EmployeeType.SALARIED:
            return SalariedEmployee()
        else:
            raise ValueError("Not a valid employee type.")

In [ ]:
factory = EmployeeFactory()
e1 = factory.create_employee(EmployeeType.HOURLY)
e2 = factory.create_employee(EmployeeType.SALARIED)

In [ ]:
print(e1.calculate_pay())
print(e2.calculate_pay())


## Mini workshop: Replacing switch statements

Restructure the following code so that a "switch statement" is used only when
creating of the objects:

In [ ]:
from dataclasses import dataclass  # noqa: E402
from abc import ABC  # noqa: E402

In [ ]:
COMPUTER_TYPE_PC = 0
COMPUTER_TYPE_MAC = 1
COMPUTER_TYPE_CHROMEBOOK = 2

In [ ]:
@dataclass
class ComputerV1:
    computer_type: int

In [ ]:
def compile_code(computer: ComputerV1):
    if computer.computer_type == COMPUTER_TYPE_PC:
        print("Compiling code for PC.")
    elif computer.computer_type == COMPUTER_TYPE_MAC:
        print("Compiling code for Mac.")
    elif computer.computer_type == COMPUTER_TYPE_CHROMEBOOK:
        print("Compiling code for Chromebook.")
    else:
        raise ValueError(f"Don't know how to compile code for {computer}.")

In [ ]:
my_pc = ComputerV1(COMPUTER_TYPE_PC)
my_mac = ComputerV1(COMPUTER_TYPE_MAC)
my_chromebook = ComputerV1(COMPUTER_TYPE_CHROMEBOOK)

In [ ]:
compile_code(my_pc)
compile_code(my_mac)
compile_code(my_chromebook)


## More rules for functions

- Use descriptive names
- Use few (or no) arguments
- Don’t use Boolean arguments (flag arguments)
- Avoid hidden side effects


## Hidden Side-Effects


```java
bool checkPassword(std::string userName, std::string password) {
    User& user = UserGateway.findByName(userName);
    if (user != User.NULL) {
        std::string codedPhrase = user.getPhraseEncodedByPassword();
        std::string phrase = cryptographer.decrypt(codedPhrase, password);
        if (phrase == "Valid Password") {
            session.initialize();
            return true;
        }
    }
    return false;
}
```


## Avoid output arguments

Python has no "real" output arguments. But modification of objects often has
similar consequences:

In [ ]:
class HitResultV1:
    pass

In [ ]:
class PlayerV1:
    def check_collision(self, obstacles, hit_result):
        # Complicated computation...
        hit_result.collision_occurred = True

In [ ]:
player = PlayerV1()
hit_result = HitResultV1()
player.check_collision([], hit_result)
if hit_result.collision_occurred:  # type: ignore
    print("Detected collision!")

In [ ]:
player = Player()
hit_result = player.check_collision([])
if hit_result.collision_occurred:
    print("Detected collision!")


## Command-Query Separation

In [ ]:
default_value = -1

In [ ]:
def bad_has_default_value() -> bool:
    global default_value
    if default_value >= 0:
        return True
    else:
        default_value = 123
        return False


## Error Reporting

Use exceptions for error reporting.

(See `module_170_exceptions`.)


## DRY: Don't Repeat Yourself

- Try to eliminate duplicated code
  - It bloats the code
  - It requires multiple modifications for every change
- But: often duplicated code is interspersed with other code
- Take into account the scope in which you keep code DRY!